In [7]:
from os import listdir

def fileids(path):
    fileList = list()
    
    path = path + ('' if path[-1] == '/' else '/')
    
    for f in listdir(path):
        if f.endswith('.txt'):
            fileList.append(path+f)
            
    return fileList

In [8]:
import re

p1 = re.compile('[a-zA-Z0-9_-]+@(?:[.]?[a-zA-Z0-9_-]+)+')
p2 = re.compile('[{}]'.format(re.escape('''"'=ㆍⓒ'"''')))
p3 = re.compile('[\(\[].+?[\)\]]')
p4 = re.compile('[a-zA-Z]+')
p5 = re.compile('\s+')

In [11]:
corpus = list()

for f in fileids(r"C:\Users\dlel2\Documents\GitHub\INISW_2023\텍스트데이터처리\news"):
    with open(f, 'r', encoding='utf8') as fp:
        corpus.append(p5.sub(' ', 
            p4.sub(' ', 
            p3.sub(' ', 
            p2.sub(' ', 
            p1.sub(' ', fp.read()))))).strip())

In [12]:
def ngram2(t, n=2):
    tokens = list(t)
    gram = list()
 
    if len(tokens) < n:
        return tokens
    for i in range(len(tokens)-(n-1)):
        gram.append(''.join(tokens[i:i+n]))
    return gram

In [14]:
gram1

Counter({'올': 178,
         '해': 3103,
         ' ': 108314,
         '여': 1197,
         '름': 85,
         '부': 1639,
         '터': 448,
         '는': 5197,
         '코': 233,
         '로': 4918,
         '나': 1102,
         '1': 1700,
         '9': 635,
         '확': 507,
         '진': 1244,
         '자': 2920,
         '격': 257,
         '리': 1523,
         '와': 692,
         '실': 690,
         '내': 1081,
         '마': 633,
         '스': 2783,
         '크': 350,
         '의': 4270,
         '무': 768,
         '조': 803,
         '치': 656,
         '가': 3746,
         '모': 837,
         '두': 562,
         '제': 1696,
         '될': 130,
         '전': 2079,
         '망': 194,
         '입': 560,
         '니': 3603,
         '다': 8860,
         '.': 7659,
         '중': 1468,
         '앙': 48,
         '재': 1431,
         '난': 420,
         '안': 1141,
         '대': 2725,
         '책': 233,
         '본': 417,
         '오': 635,
         '늘': 149,
         '방': 755,
         '역': 532,
       

In [15]:
gram2

Counter({'올해': 55,
         '해 ': 1222,
         ' 여': 353,
         '여름': 6,
         '름부': 2,
         '부터': 168,
         '터는': 12,
         '는 ': 4847,
         ' 코': 115,
         '코로': 43,
         '로나': 46,
         '나1': 30,
         '19': 146,
         '9로': 4,
         '로 ': 3772,
         ' 확': 424,
         '확진': 4,
         '진자': 6,
         '자 ': 1424,
         ' 격': 38,
         '격리': 8,
         '리와': 19,
         '와 ': 595,
         ' 실': 255,
         '실내': 3,
         '내마': 1,
         '마스': 9,
         '스크': 47,
         '크 ': 69,
         ' 의': 510,
         '의무': 26,
         '무 ': 49,
         ' 조': 486,
         '조치': 27,
         '치가': 20,
         '가 ': 1615,
         ' 모': 617,
         '모두': 269,
         '두 ': 123,
         ' 해': 735,
         '해제': 11,
         '제될': 2,
         '될 ': 114,
         ' 전': 1414,
         '전망': 38,
         '망입': 5,
         '입니': 198,
         '니다': 3182,
         '다.': 5643,
         '.코': 4,
         '9 ': 46,
         ' 중

In [16]:
gram3

Counter({'올해 ': 49,
         '해 여': 6,
         ' 여름': 5,
         '여름부': 2,
         '름부터': 2,
         '부터는': 5,
         '터는 ': 12,
         '는 코': 9,
         ' 코로': 38,
         '코로나': 43,
         '로나1': 30,
         '나19': 30,
         '19로': 4,
         '9로 ': 4,
         '로 확': 224,
         ' 확진': 4,
         '확진자': 4,
         '진자 ': 4,
         '자 격': 3,
         ' 격리': 7,
         '격리와': 1,
         '리와 ': 19,
         '와 실': 1,
         ' 실내': 3,
         '실내마': 1,
         '내마스': 1,
         '마스크': 8,
         '스크 ': 16,
         '크 의': 1,
         ' 의무': 23,
         '의무 ': 10,
         '무 조': 1,
         ' 조치': 25,
         '조치가': 5,
         '치가 ': 19,
         '가 모': 8,
         ' 모두': 269,
         '모두 ': 48,
         '두 해': 2,
         ' 해제': 11,
         '해제될': 1,
         '제될 ': 2,
         '될 전': 3,
         ' 전망': 37,
         '전망입': 5,
         '망입니': 5,
         '입니다': 193,
         '니다.': 2625,
         '다.코': 4,
         '.코로': 2,
         '19 ': 24,
      

In [17]:
gram4

Counter({'올해 여': 1,
         '해 여름': 2,
         ' 여름부': 2,
         '여름부터': 2,
         '름부터는': 1,
         '부터는 ': 5,
         '터는 코': 1,
         '는 코로': 5,
         ' 코로나': 38,
         '코로나1': 30,
         '로나19': 30,
         '나19로': 4,
         '19로 ': 4,
         '9로 확': 1,
         '로 확진': 1,
         ' 확진자': 4,
         '확진자 ': 4,
         '진자 격': 3,
         '자 격리': 3,
         ' 격리와': 1,
         '격리와 ': 1,
         '리와 실': 1,
         '와 실내': 1,
         ' 실내마': 1,
         '실내마스': 1,
         '내마스크': 1,
         '마스크 ': 5,
         '스크 의': 1,
         '크 의무': 1,
         ' 의무 ': 8,
         '의무 조': 1,
         '무 조치': 1,
         ' 조치가': 4,
         '조치가 ': 5,
         '치가 모': 1,
         '가 모두': 4,
         ' 모두 ': 48,
         '모두 해': 2,
         '두 해제': 1,
         ' 해제될': 1,
         '해제될 ': 1,
         '제될 전': 1,
         '될 전망': 3,
         ' 전망입': 5,
         '전망입니': 5,
         '망입니다': 5,
         '입니다.': 185,
         '니다.코': 2,
         '다.코로': 2,
         '.코로나

In [21]:
gram_finder[3].items()

dict_items([('올해 ', 49), ('해 여', 6), (' 여름', 5), ('여름부', 2), ('름부터', 2), ('부터는', 5), ('터는 ', 12), ('는 코', 9), (' 코로', 38), ('코로나', 43), ('로나1', 30), ('나19', 30), ('19로', 4), ('9로 ', 4), ('로 확', 224), (' 확진', 4), ('확진자', 4), ('진자 ', 4), ('자 격', 3), (' 격리', 7), ('격리와', 1), ('리와 ', 19), ('와 실', 1), (' 실내', 3), ('실내마', 1), ('내마스', 1), ('마스크', 8), ('스크 ', 16), ('크 의', 1), (' 의무', 23), ('의무 ', 10), ('무 조', 1), (' 조치', 25), ('조치가', 5), ('치가 ', 19), ('가 모', 8), (' 모두', 269), ('모두 ', 48), ('두 해', 2), (' 해제', 11), ('해제될', 1), ('제될 ', 2), ('될 전', 3), (' 전망', 37), ('전망입', 5), ('망입니', 5), ('입니다', 193), ('니다.', 2625), ('다.코', 4), ('.코로', 2), ('19 ', 24), ('9 중', 2), (' 중앙', 34), ('중앙재', 1), ('앙재난', 1), ('재난안', 1), ('난안전', 1), ('안전대', 1), ('전대책', 1), ('대책본', 2), ('책본부', 4), ('본부는', 2), ('부는 ', 83), ('는 오', 16), (' 오늘', 41), ('오늘 ', 37), ('늘 방', 1), (' 방역', 28), ('방역 ', 11), ('역 의', 4), (' 의료', 18), ('의료체', 4), ('료체계', 4), ('체계의', 1), ('계의 ', 15), ('의 일', 17), (' 일상', 8), ('일상 ', 4), ('상 회', 5), (' 회복

In [24]:
k = s[:3]
freq2 = gram_finder[4]

In [27]:
k

'가족관'

In [26]:
k[0]

'가'

In [28]:
dict(filter(lambda r:re.match(k, r[0]), freq2.items()))

{'가족관계': 2}

In [30]:
 c = dict(filter(lambda r:re.match(k, r[0]), freq2.items()))

In [34]:
c

{'가족관계': 2}

In [39]:
t = max(c, key=c.get)
t

'가족관계'

In [38]:
f = c[t]
f

2

In [40]:
from collections import Counter

gram1 = Counter(ngram2('\n'.join(corpus),1))
gram2 = Counter(ngram2('\n'.join(corpus),2))
gram3 = Counter(ngram2('\n'.join(corpus),3))
gram4 = Counter(ngram2('\n'.join(corpus),4))

gram_finder = {1:gram1, 2:gram2, 3:gram3, 4:gram4}

def auto_space(s, base=2, verbose=True):
    result = ''
    
    if base not in [2,3,4]:
        return s
    
    if len(s) < base:
        return s
    
    freq1 = gram_finder[base-1]
    freq2 = gram_finder[base]
    
    i = base-1
    k = s[:i]
    while i < len(s):
        c = dict(filter(lambda r:re.match(k, r[0]), freq2.items()))
        print(c)
        result += k[0]
        print(result)
        if k not in freq1:
            if verbose:
                print('Not found:', k, 'in Freq1')
            i += 1
            k = k[1:] + s[i-1]
            continue
            
        if len(c) == 0:
            if verbose:
                print('Not found:', k, 'in Freq2')
            i += 1
            k = k[1:] + s[i-1]
            continue
        
        t = max(c, key=c.get)
        f = c[t]
        
        if verbose:
            print('i:{} | K:{} | P:{:.2f} - [{}]'.format(i, k, f/freq1[k], t))
        
        if t[-1] == ' ':
            k = t[1:]
        else:
            i += 1
            k = k[1:] + s[i-1]
            
    result += s[i-base+1:]
    
    return result

s = '가족관계등록법은 가정폭력 피해자가 가해자로 지정한 배우자 직계혈족이 피해자 본인의 가족관계증명서 등을 받아 갈 수 없도록 못 박았다.'
s = re.sub(' ', '', s)
auto_space(s, 4, False)

{'가족관계': 2}
가
{'족관계등': 2}
가족
{'관계등록': 2}
가족관
{'계등록 ': 1, '계등록부': 1}
가족관계
{'등록 고': 2, '등록 업': 1, '등록 제': 1, '등록 당': 1, '등록 건': 1, '등록 때': 1}
가족관계등
{}
가족관계등록
{' 법은 ': 3}
가족관계등록 
{'법은 남': 1, '법은 주': 1, '법은 대': 1, '법은 윤': 1, '법은 존': 1, '법은 야': 1, '법은 반': 2, '법은 우': 1, '법은 농': 1, '법은 세': 1, '법은 통': 2, '법은 추': 1, '법은 오': 1}
가족관계등록 법
{'은 가파': 1, '은 가격': 3, '은 가 ': 2, '은 가상': 1, '은 가족': 4, '은 가짜': 6, '은 가해': 2, '은 가시': 2}
가족관계등록 법은
{' 가정에': 3, ' 가정환': 1, ' 가정폭': 1, ' 가정사': 1}
가족관계등록 법은 
{'가정폭력': 1}
가족관계등록 법은 가
{'정폭력 ': 1}
가족관계등록 법은 가정
{'폭력 넘': 10, '폭력 피': 1, '폭력 관': 3, '폭력 진': 3, '폭력 대': 2}
가족관계등록 법은 가정폭
{'력 피했': 1}
가족관계등록 법은 가정폭력
{' 피해 ': 29, ' 피해자': 19, ' 피해를': 7, ' 피해가': 7, ' 피해로': 1, ' 피해보': 1, ' 피해배': 1}
가족관계등록 법은 가정폭력 
{'피해 정': 1, '피해 실': 1, '피해 방': 4, '피해 복': 2, '피해 배': 1, '피해 학': 2, '피해 없': 2, '피해 예': 3, '피해 신': 2, '피해 눈': 2, '피해 사': 10, '피해 건': 3, '피해 규': 1}
가족관계등록 법은 가정폭력 피
{'해 자진': 2, '해 자식': 1, '해 자국': 2, '해 자연': 1}
가족관계등록 법은 가정폭력 피해
{}
가족관계등록 법은 가정폭력 피해 
{}
가족관계등록 법은 가정폭력 피해 자
{}


'가족관계등록 법은 가정폭력 피해 자가가해자로지정한 배우자 직계혈족이피해자본인의 가족 관계증명서등을받아갈수없도록 못박았다.'

dict가 저장되지 않은 key를 만나면 

In [ ]:
'가족관계등록 법은 가정폭력 피해 자가가해자로지정한 배우자 직계혈족이피해자본인의 가족 관계증명서등을받아갈수없도록 못박았다.'